In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from factue.methods.llm_calls import make_call
from factue.methods.llm_langchain.llm import Llm
from factue.utils.types import ModelMode, ModelName, ModelProvider

In [3]:
t = """
The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More None
"""
# t = 'car'

In [4]:
llm = Llm(
    model_name=ModelName.LLAMA_31_8B,
    provider=ModelProvider.OLLAMA,
    mode=ModelMode.CHAT,
    temperature=0.8,
    num_predict=15,
    seed=-1,
)

make_call(llm, "extract_task", "extract_001", {"text": t}, 5)

******************** -1


['The Karnofsky Jewish family employed and adopted into their home a 7',
 'The Karnofsky Jewish family employed a 7-year-old boy and adopted',
 'The Karnofsky Jewish family employed and adopted a starving 7-year-old',
 'The Karnofsky family employed and adopted a 7-year-old starving boy',
 'The Karnofsky Jewish family employed and adopted a 7-year-old starving']

In [5]:
from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp
import pandas as pd

In [6]:
sample_in = (
    PROJECT_ROOT / "data/01_preprocessed" / "dev" / "dev-eng" / "batch_0000.parquet"
)
df = pd.read_parquet(sample_in).head(3)
df

,text,reference,text_lang,text_lang_score,reference_lang,reference_lang_score
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get f...",Photo shows Louis Armstrong as a child,en,1.00,en,0.95
1,Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill\nCo\nThis little guy was rescued after his\nmother was murdered by a hunter.\nHis eyes say it a...,This leopard cub's mother was killed by a trophy hunter,en,0.94,en,0.99
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,uk,0.10,en,0.66


In [7]:
make_call(llm, "extract_task", "extract_001", {"text": t}, 3)

['The Karnofsky Jewish family employed and cared for the boy from age',
 'The Karnofsky Jewish family employed a 7-year-old boy and gave',
 'The Karnofsky family employed a 7-year-old boy and adopted him']

In [11]:
df["claim"] = df.apply(
    lambda row: make_call(
        llm=llm,
        step_id="extract_task",
        prompt_id="extract_001",
        variables={"text": row["text"]},
        max_iterations=5,
    ),
    axis=1,
)
[i for x in df.claim.values for i in x]

['The Karnofsky Jewish family employed and adopted into their home a 7',
 'The Karnofsky Jewish family employed and adopted a 7-year-old starving',
 'The Karnofsky Jewish family employed and adopted a 7-year-old starving',
 'The Karnofsky family employed and adopted a starving 7-year-old boy',
 'The Karnofsky family employed and adopted a starving 7-year-old boy',
 'A mother orca was murdered by a hunter, resulting in the rescue of',
 'A mother of this orphaned animal was murdered by a hunter.',
 'A mother of this little guy was murdered by a hunter.',
 'A mother of this orphaned animal was murdered by a hunter.',
 'A mother of this young animal was murdered by a hunter.',
 'A heavy rain alert has been issued in Hyderabad.',
 'A heavy rain alert has been issued for Hyderabad due to a possible crocodile',
 'A heavy rain alert has been issued for Hyderabad.',
 'Heavy rain is alerting for a crocodile in Hyderabad.',
 'A heavy rain alert has been issued in Hyderabad.']

['The Karnofsky family employed a 7-year-old starving boy and gave',
 'The Karnofsky Jewish family employed and adopted a 7-year-old starving',
 'The Karnofsky Jewish family employed and adopted a 7-year-old boy',
 'The Karnofsky family employed and supported a starving 7-year-old boy',
 'The Karnofsky Jewish family employed and adopted a 7-year-old boy',
 'A mother of this little guy was murdered by a hunter.',
 'A mother of this orphaned animal was murdered by a hunter.',
 'A hunter murdered the mother of this rescued animal, highlighting the devastating impact of',
 'A mother of this little animal was murdered by a hunter.',
 'A mother of this animal was murdered by a hunter, resulting in the need',
 'A heavy rain alert has been issued in Hyderabad.',
 'A heavy rain alert has been issued for Hyderabad due to a possible crocodile',
 'A crocodile has been spotted in Hyderabad due to the heavy rain.',
 'A heavy rain alert has been issued in Hyderabad due to the presence of a',
 'Heavy 